importar dados

In [392]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [393]:
biog = pd.read_csv("biodegradable_a.csv", sep=",")

X = biog.values[:, 0:41]
y = biog["Biodegradable"].values

# X_train, X_IVS, y_train, y_IVS = train_test_split(X, y, test_size=0.33, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

X_cols = biog.columns[0:41]

# pd.DataFrame(X, columns=X_cols)

Check missing values

In [394]:
# check number of missing values in each column.

# dataset without lines with NaN values
biog_nan = biog.dropna()
#print(len(biog_nan))

print("No. of rows original dataset = {}".format(len(biog)))
print("No. of rows without NaN = {} ({}%)".format(len(biog_nan), round(len(biog_nan) / len(biog) * 100)))


# check number of missing values in each column.
missValues = biog.isna().sum()

missValues = missValues.to_frame()
print("Threshold number of missing values per column = {} (15%)".format(str(round(0.15 * len(X)))))

# print(missValues)

No. of rows original dataset = 4564
No. of rows without NaN = 889 (19%)
Threshold number of missing values per column = 685 (15%)


escalar e normalizar os dados (POWER TRANSFORMER)

In [395]:
from sklearn.preprocessing import PowerTransformer

scaler = PowerTransformer()
XscaledPT = scaler.fit_transform(X_train)

df1 = pd.DataFrame(XscaledPT, columns=X_cols)

# for i in range(len(Xscaled[0])):
#     values = Xscaled[:, i]
#     plt.hist(values)
#     plt.title(X_cols[colsToScale[i]])
#     plt.show()
#     plt.close()

Escalar os dados (Standard Scaler)

In [396]:
import statistics

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
XscaledStS = scaler.fit_transform(X_train)

df2 = pd.DataFrame(XscaledStS, columns=X_cols)

# for i in range(len(XscaledStS[0])):
#     values = XscaledStS[:, i]
#     plt.hist(values)
#     plt.title(X_cols[i] + " " + str(i))
#     plt.show()
#     plt.close()

Metodo de Imputação

In [397]:
useMean = [0, 1, 7, 11, 12, 13, 14, 21, 26, 27, 38]
useMedian = np.setdiff1d(range(41), useMean)

imputar dados

In [398]:
# Impute missing values
from sklearn.impute import SimpleImputer 

# strategy = mean or median (for numerical data)
imputerMean = SimpleImputer(missing_values = np.nan, strategy = "mean") 
imputerMode = SimpleImputer(missing_values = np.nan, strategy = "median") 

#Standard Scaled
XscaledStSInputed1 = imputerMean.fit_transform(XscaledStS[:, useMean])
XscaledStSInputed2 = imputerMean.fit_transform(XscaledStS[:, useMedian])

df1 = pd.DataFrame(XscaledStSInputed1, columns=X_cols[useMean])
df2 = pd.DataFrame(XscaledStSInputed2, columns=X_cols[useMedian])
df3 = pd.concat([df1, df2], axis=1)

XscaledStSInputed = df3.to_numpy()

# Feature selection

Correlation

In [399]:
N, M = X_train.shape

v = np.hstack((y_train.reshape((N, 1)), XscaledStSInputed))
v[v == "RB"] = 1.0
v[v == "NRB"] = 0.0

pd.DataFrame(v)
corrMatrix = pd.DataFrame((np.corrcoef(v.T.astype(float))), columns=X_cols.insert(0, "Y")).iloc[0].sort_values()[0:10]

selectedCorr = list(corrMatrix.index)

print(corrMatrix)
print("The 10 selected features are:", selectedCorr)

SdssC      -0.490899
nHDon      -0.434869
F02_CN     -0.426170
Psi_i_1d   -0.389473
F03        -0.388530
SM6_B      -0.386957
nN_N       -0.386074
LOC        -0.366294
F03_CO     -0.359854
SpMax_L    -0.334098
Name: 0, dtype: float64
The 10 selected features are: ['SdssC', 'nHDon', 'F02_CN', 'Psi_i_1d', 'F03', 'SM6_B', 'nN_N', 'LOC', 'F03_CO', 'SpMax_L']


Random Forest Feature Selection

In [400]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

y_train_temp = y_train
y_train_temp[y_train_temp == "RB"] = 1.0
y_train_temp[y_train_temp == "NRB"] = 0.0

sel = SelectFromModel(RandomForestClassifier(), max_features=10)
sel.fit(XscaledStSInputed, y_train_temp.astype(float))

selectedForest = []
selectedForestIndixes = []

for feature_list_index in sel.get_support(indices=True):
    selectedForestIndixes.append(feature_list_index)
    selectedForest.append(X_cols[feature_list_index])
    print("Feature: ", X_cols[feature_list_index], "| Importance: ", sel.estimator_.feature_importances_[feature_list_index])

Feature:  SpMax_L | Importance:  0.060934994393037324
Feature:  C | Importance:  0.045813128953902625
Feature:  F03 | Importance:  0.040609544328645525
Feature:  SdssC | Importance:  0.10223798649881853
Feature:  LOC | Importance:  0.061254049909900035
Feature:  SM6_L | Importance:  0.048937360551812255
Feature:  F03_CO | Importance:  0.04173332785133932
Feature:  nN_N | Importance:  0.053893026958912994
Feature:  nHDon | Importance:  0.052933421370986715
Feature:  nX | Importance:  0.048095514208176975


StepWise (Forward)

In [401]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.tree import DecisionTreeClassifier

N,M = XscaledStSInputed.shape

dtc = DecisionTreeClassifier()
sfs = SequentialFeatureSelector(dtc, n_features_to_select=10, direction="forward")

sfs.fit(XscaledStSInputed, y_train_temp.astype(float))

features = sfs.get_support()
ST_for_selected_features = np.arange(M)[features]

print(X_cols[ST_for_selected_features])

Index(['SdssC', 'SM6_L', 'F03_CO', 'Mi', 'nN_N', 'nArNO2', 'nCIR', 'TI2_L',
       'nHDon', 'SpMax_B'],
      dtype='object')


StepWise (BackWard)

In [402]:
dtc = DecisionTreeClassifier()
sfs = SequentialFeatureSelector(dtc, n_features_to_select=10, direction="backward")

sfs.fit(XscaledStSInputed, y_train_temp.astype(float))

features = sfs.get_support()
ST_back_selected_features = np.arange(M)[features]

print(X_cols[ST_back_selected_features])

Index(['SpMax_L', 'nHM', 'F01', 'F03', 'SdssC', 'F03_CO', 'Me', 'C_026',
       'nHDon', 'nN'],
      dtype='object')


Indexes of Features Selcted

In [403]:
corr_index = [df3.columns.get_loc(f) for f in selectedCorr]
rand_forest_index = selectedForestIndixes
step_for_index = ST_for_selected_features
step_back_index = ST_back_selected_features

print(corr_index)
print(rand_forest_index)
print(step_for_index)
print(step_back_index)

[3, 35, 34, 9, 18, 10, 22, 5, 19, 0]
[0, 7, 10, 11, 13, 14, 15, 18, 34, 40]
[11 14 15 17 18 19 22 30 34 35]
[ 0  2  3 10 11 15 16 32 34 37]


In [417]:
selec_feat_indexes = [corr_index,
                      rand_forest_index,
                      step_for_index,
                      step_back_index]

selec_feat_names = ["Correlation",
                      "Random Forest",
                      "StepWise For",
                      "StepWise Back"]


Define classification metrics

In [420]:
from sklearn.metrics import precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix, accuracy_score

def printClassResults(truth, preds):
    print("The Accuracy is: %7.4f" % accuracy_score(truth, preds))
    print("The Precision is: %7.4f" % precision_score(truth, preds))
    print("The Recall is: %7.4f" % recall_score(truth, preds))
    print("The F1 score is: %7.4f" % f1_score(truth, preds))
    print("The Matthews correlation coefficient is: %7.4f" % matthews_corrcoef(truth, preds))
    print()
    print("This is the Confusion Matrix")
    print(pd.DataFrame(confusion_matrix(truth, preds)))
    print("\n")

# printClassResults(y, preds)

# First Model

KNN

In [444]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


def gaussian(dsts):
    kernel_width = .5
    weights = np.exp(-(dsts**2)/kernel_width)
    return weights


for i in range(4):
    print("Feature selection method: " + selec_feat_names[i])

    X_train, X_test, y_train, y_test = train_test_split(
        XscaledStSInputed[:, selec_feat_indexes[i]], y_train_temp.astype(float), test_size=0.33, random_state=0)

    accsGaus = []
    accsWeig = []
    accs = []

    for k in range(2, 10, 2):
        knn = KNeighborsClassifier(n_neighbors=k, weights=gaussian)
        knn.fit(X_train, y_train)
        preds = knn.predict(X_test)
        accsGaus.append(accuracy_score(y_test, preds))
        print("Nº Neighbours " + str(k) + " | weights=gaussian" + "\n")
        printClassResults(y_test, preds)

        knn1 = KNeighborsClassifier(n_neighbors=k, weights="distance")
        knn1.fit(X_train, y_train)
        preds = knn1.predict(X_test)
        accsWeig.append(accuracy_score(y_test, preds))
        print("Nº Neighbours " + str(k) + " | weights=distance" + "\n")
        printClassResults(y_test, preds)

        knn2 = KNeighborsClassifier(n_neighbors=k)
        knn2.fit(X_train, y_train)
        preds = knn2.predict(X_test)
        accs.append(accuracy_score(y_test, preds))
        print("Nº Neighbours " + str(k) + " | no weights" + "\n")
        printClassResults(y_test, preds)

    # plt.plot(range(1, 10, 2), accsGaus, label="gaussian")
    # plt.plot(range(1, 10, 2), accsWeig, label="distance")
    # plt.plot(range(1, 10, 2), accs, label="no weights")
    # plt.legend()
    # plt.grid()
    # plt.show()

Feature selection method: Correlation
Nº Neighbours 2 | weights=gaussian

The Accuracy is:  0.9475
The Precision is:  0.9564
The Recall is:  0.9823
The F1 score is:  0.9692
The Matthews correlation coefficient is:  0.7942

This is the Confusion Matrix
     0    1
0  122   38
1   15  834


Nº Neighbours 2 | weights=distance

The Accuracy is:  0.9475
The Precision is:  0.9564
The Recall is:  0.9823
The F1 score is:  0.9692
The Matthews correlation coefficient is:  0.7942

This is the Confusion Matrix
     0    1
0  122   38
1   15  834


Nº Neighbours 2 | no weights

The Accuracy is:  0.9326
The Precision is:  0.9688
The Recall is:  0.9505
The F1 score is:  0.9596
The Matthews correlation coefficient is:  0.7585

This is the Confusion Matrix
     0    1
0  134   26
1   42  807


Nº Neighbours 4 | weights=gaussian

The Accuracy is:  0.9455
The Precision is:  0.9501
The Recall is:  0.9870
The F1 score is:  0.9682
The Matthews correlation coefficient is:  0.7841

This is the Confusion Matri

Random Forest

In [423]:
for i in range(4):
        print("Feature selection method: " + selec_feat_names[i] + "\n")
        X_train, X_test, y_train, y_test = train_test_split(
                XscaledStSInputed[:, selec_feat_indexes[i]], y_train_temp.astype(float), test_size=0.33, random_state=0)

        rf = RandomForestClassifier(n_estimators=50)
        rf.fit(X_train, y_train)

        preds = rf.predict(X_test)

        printClassResults(y_test, preds)

Feature selection method: Correlation

The Accuracy is:  0.9376
The Precision is:  0.9507
The Recall is:  0.9764
The F1 score is:  0.9634
The Matthews correlation coefficient is:  0.7546

This is the Confusion Matrix
     0    1
0  117   43
1   20  829


Feature selection method: Random Forest

The Accuracy is:  0.9495
The Precision is:  0.9640
The Recall is:  0.9764
The F1 score is:  0.9702
The Matthews correlation coefficient is:  0.8059

This is the Confusion Matrix
     0    1
0  129   31
1   20  829


Feature selection method: StepWise For

The Accuracy is:  0.9554
The Precision is:  0.9674
The Recall is:  0.9800
The F1 score is:  0.9737
The Matthews correlation coefficient is:  0.8288

This is the Confusion Matrix
     0    1
0  132   28
1   17  832


Feature selection method: StepWise Back

The Accuracy is:  0.9475
The Precision is:  0.9639
The Recall is:  0.9741
The F1 score is:  0.9690
The Matthews correlation coefficient is:  0.7990

This is the Confusion Matrix
     0    1
0

Bagging Classifier

In [434]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier, BaggingRegressor
from sklearn.linear_model import LinearRegression

for n in range(2, 10, 2):
    print("Nº of estimators:" + " " + str(n))
    bag = BaggingClassifier(base_estimator=GaussianNB(),
                            n_estimators=n, random_state=22)

    for i in range(4):
        print("Feature selection method: " + selec_feat_names[i] + "\n")

        X_train, X_test, y_train, y_test = train_test_split(
            XscaledStSInputed[:, selec_feat_indexes[i]], y_train_temp.astype(float), test_size=0.33, random_state=0)

        bag.fit(X_train, y_train)

        preds = bag.predict(X_test)
        
        printClassResults(y_test, preds)

Nº of estimators: 2
Feature selection method: Correlation

The Accuracy is:  0.8821
The Precision is:  0.8883
The Recall is:  0.9835
The F1 score is:  0.9335
The Matthews correlation coefficient is:  0.4736

This is the Confusion Matrix
    0    1
0  55  105
1  14  835


Feature selection method: Random Forest

The Accuracy is:  0.9177
The Precision is:  0.9517
The Recall is:  0.9505
The F1 score is:  0.9511
The Matthews correlation coefficient is:  0.6925

This is the Confusion Matrix
     0    1
0  119   41
1   42  807


Feature selection method: StepWise For

The Accuracy is:  0.9237
The Precision is:  0.9289
The Recall is:  0.9847
The F1 score is:  0.9560
The Matthews correlation coefficient is:  0.6880

This is the Confusion Matrix
    0    1
0  96   64
1  13  836


Feature selection method: StepWise Back

The Accuracy is:  0.9118
The Precision is:  0.9358
The Recall is:  0.9611
The F1 score is:  0.9483
The Matthews correlation coefficient is:  0.6517

This is the Confusion Matrix